In [ ]:
# This cell contains the parameters. It should have the tag "parameters"
SRCDIR = "."
patch_size = 256 #options 256 or 128
overlap = 0 #options 0 or half patch size ?
nb_patches = 500 #options between 1 and 1000

In [ ]:
img0 = 'input_0.png'
img1 = 'input_1.png'
img2 = 'input_2.png'

In [ ]:
import os
import sys
sys.path.append(SRCDIR)

In [ ]:
from IPython.display import display, Image
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import imageio

#import sys
#sys.path.append('src_updated')
import src_updated.forensic_similarity as forsim #forensic similarity tool
from src_updated.utils.blockimage import tile_image #function to tile image into blocks

In [ ]:
def output(img0,img1,img2, patch_size, overlap, nb_patches):
    
    print(nb_patches)
    ### READ IMAGES
    
    I0 = imageio.imread(img0)
    I1 = imageio.imread(img1)
    I2 = imageio.imread(img2)
    
    ### EXTRACT PATCHES
    overlap = int(overlap*patch_size)
    np.random.seed(0)
    #tiles and xy coordinates of each tile for image 0
    T0,xy0 = tile_image(I0,width=patch_size,height=patch_size, 
                x_overlap=overlap,y_overlap=overlap)
    #tiles and xy coordinates of each tile for image 1
    T1,xy1 = tile_image(I1,width=patch_size,height=patch_size,
                x_overlap=overlap,y_overlap=overlap)
    #tiles and xy coordinates of each tile for image 2
    T2,xy2 = tile_image(I2,width=patch_size,height=patch_size,
                x_overlap=overlap,y_overlap=overlap)
    
    ### SELECT PATCHES
    inds0 = np.random.randint(0,len(T0),size=nb_patches) #select random indices
    inds1 = np.random.randint(0,len(T1),size=nb_patches) 
    inds2 = np.random.randint(0,len(T2),size=nb_patches) 
    X0 = T0[inds0] #vector of randomly selected image tiles
    X1 = T1[inds1] #vector of randomly selected image tiles
    X2 = T2[inds2] #vector of randomly selected image tiles
    
    ### READ PRE-TRAINED WEIGHTS
    if patch_size == 256:
        f_weights = 'src_updated/pretrained/cam_256/-30'
        #tf.print(f_weights)
    elif patch_size == 128:
        f_weights = 'src_updated/pretrained/cam_128/-30'
    else:
        print('Incorrect patch size')
    
    ### COMPUTE FORENSIC SIMILARITY
    sim1 = forsim.calculate_forensic_similarity(X0,X1,f_weights,patch_size)
    sim2 = forsim.calculate_forensic_similarity(X0,X2,f_weights,patch_size)
    
    ### PLOT HISTOGRAM
    fig,ax = plt.subplots(1)
    ax.hist(sim1,50, alpha=0.5, label = 'Reference Image - Image 1')
    ax.hist(sim2,50, alpha=0.5,label= 'Reference Image - Image 2')
    ax.legend(bbox_to_anchor=(1.04,1), borderaxespad=0)
    ax.set_title(f'Forensic similarity for {nb_patches}')
    plt.savefig('output.png', bbox_inches='tight')
    plt.show()
    plt.close(fig)
    #return sim1, sim2

In [ ]:
#| output: false
output(img0,img1, img2, patch_size, overlap, nb_patches)

::: {.panel-tabset}
### Reference image

In [ ]:
display(Image(img0))

### Image 1

In [ ]:
display(Image(img1))

### Image 2

In [ ]:
display(Image(img2))

### Output

In [ ]:
display(Image("output.png"))

:::